In [1]:
!pip install datasets --quiet
!pip install tensorflow==2.15  --quiet
!pip install -U tf_keras  --quiet # Keras 2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 34.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
from datasets import load_dataset

import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [3]:
# mount Google Drive for persistent storage
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ls /content/gdrive/My\ Drive/MIDS/266_NLP/Final_Project_266/SQUAD2_finetune/model_checkpoints

## Helper Functions

## Load Dataset

In [14]:
from datasets import get_dataset_split_names
# check the split available
get_dataset_split_names("rajpurkar/squad_v2")

['train', 'validation']

In [5]:
## Load full data set ##
# number of rows 142,192
# size of downloaded dataset 17.7 MB
datasets = load_dataset("rajpurkar/squad_v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
## Load smaple data set ##
# sample = load_dataset("rajpurkar/squad_v2", split="train[:5000]")
# sample = sample.train_test_split(test_size=0.2)

## Preprocess Data

Still cannot handle not answerable questions, answers for those questions will have empty text and answer_start.

In [6]:
# Other checkpoints(discriminator) available
# google/electra-base-discriminator
# google/electra-large-discriminator
checkpoint = "google/electra-small-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
def check_answer(tokenized_examples, sample_id=0):
  '''
  Get answer texts from tokenized data.
  '''
  start_token =  tokenized_examples["start_positions"][sample_id]
  end_token = tokenized_examples["end_positions"][sample_id]
  if (start_token != 0) and (end_token != 0):
    return tokenizer.decode(tokenized_examples["input_ids"][sample_id][start_token:end_token+1])
  else:
    return "This question is unanswerable from the context."

In [8]:
# reference https://keras.io/examples/nlp/question_answering/
# include return over flowing tokens

def preprocess_function(examples, max_length=400, doc_stride=100):
    '''
    tokenize and truncate input, map the start and end tokens of the answer to
    the context.
    '''

    # Tokenize our examples with truncation and padding, but keep the overflows using a
    # stride. This results in one example possible giving several features when a context is long,
    # each of those features having a context that overlaps a bit the context of the previous
    # feature.
    examples["question"] = [q.lstrip() for q in examples["question"]]
    examples["context"] = [c.lstrip() for c in examples["context"]]
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a
    # map from a feature to its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original
    # context. This will help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what
        # is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this
        # span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the
            # CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the
                # answer.
                # Note: we could go after the last offset if the answer is the last word (edge
                # case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [10]:
tokenized_dataset = datasets.map(preprocess_function, batched=True, remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [11]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [12]:
model = TFAutoModelForQuestionAnswering.from_pretrained(checkpoint)

tf_model.h5:   0%|          | 0.00/54.5M [00:00<?, ?B/s]

Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForQuestionAnswering: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    shuffle=True,
    batch_size=16
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_dataset["validation"],
    shuffle=False,
    batch_size=16
)

In [16]:
# reference Table 7 in ICLR 2020 paper
# reference https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/PolynomialDecay

learning_rate_fn = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=3e-4,
    decay_steps=4000,
    end_learning_rate = 1e-5,
    cycle=False,
    power=1.0
    )

optimizer = keras.optimizers.Adam(
    learning_rate=learning_rate_fn,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-06,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_overwrite_frequency=None,
    name='small_optimizer'
)

In [17]:
# model checkpoint callback

checkpoint_path = '/content/gdrive/My Drive/MIDS/266_NLP/Final_Project_266/SQUAD2_finetune/model_checkpoints' + '_{epoch:02d}-{val_start_logits_accuracy:.2f}-{val_end_logits_accuracy:.2f}'

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='accuracy',
    mode='max',
    save_best_only=False,
    save_freq="epoch"
    )

In [18]:
model.compile(optimizer=optimizer,
              metrics=['accuracy']
              )


model.fit(tf_train_set,
          validation_data=tf_validation_set,
          batch_size=32,
          epochs=2,
          callbacks=[model_checkpoint_callback])

Epoch 1/2


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
8217/8217 [==============================] - 1095s 125ms/step - loss: 1.5518 - end_logits_accuracy: 0.5465 - start_logits_accuracy: 0.5303 - val_loss: 1.2146 - val_end_logits_accuracy: 0.6238 - val_start_logits_accuracy: 0.6158
Epoch 2/2
8217/8217 [==============================] - 1011s 123ms/step - loss: 1.1340 - end_logits_accuracy: 0.6597 - start_logits_accuracy: 0.6382 - val_loss: 1.2704 - val_end_logits_accuracy: 0.6172 - val_start_logits_accuracy: 0.6107
